In [ ]:
#| default_exp photomapper

# PhotoMapper

> API details.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import base64
from os.path import exists
from pathlib import Path

from exif import Image
import folium
from folium import IFrame

import matplotlib
import numpy as np
import pandas as pd
import PIL

In [ ]:
#| export
def coords_to_decimal(coords: tuple[float,float],  # (45.5454545, -115.12312312)
                      ref: str  # "N", "S", "E", "W"
                     ) -> float:
    """Covert a tuple of coordinates in the format (degrees, minutes, seconds)
    and a reference to a decimal representation.
    
    Returns:
        float: A signed float of decimal representation of the coordinate.
    """
    if ref.upper() in ['W', 'S']:
        mul = -1
    elif ref.upper() in ['E', 'N']:
        mul = 1
    else:
        print(f"Expecting one of 'N', 'S', 'E' or 'W', got {ref} instead.")
        
    return mul * (coords[0] + coords[1] / 60 + coords[2] / 3600) 

In [ ]:
#| export
def get_decimal_coord_from_exif(exif_data: Image  # takes an Exif Image
                               ) -> tuple[float, float, float]:
    """Get coordinate data from exif and convert to a tuple of 
    decimal latitude, longitude and altitude.
    Returns:
        tuple[float, float, float]: A tuple of decimal coordinates (lat, lon, alt)
    """
    try:
        lat = coords_to_decimal(
            exif_data['gps_latitude'], 
            exif_data['gps_latitude_ref']
            )
        lon = coords_to_decimal(
            exif_data['gps_longitude'], 
            exif_data['gps_longitude_ref']
            )
        alt = exif_data['gps_altitude']
        return (lat, lon, alt)
    except (AttributeError, KeyError):
        print('Image does not contain spatial data or data is invalid.')  
        raise

In [ ]:
#| export
def get_gps_data_from_images(image_path: Path,  # "Path object"
                             image_extension: str = '*.jpg'  # image extensions glob
                            ) -> dict[str, dict]:
    """Create a dictionary of gps data from images in a directory.
    Returns:
        dict[str, dict]: A dictionary with filename as the key
            and a value of a dictionary if the format:
                {
                    'coordinates': tuple[float, ...],
                    'timestamp': str
                }
    """
    coord_dict = dict()
    source_files = [f for f in image_path.rglob(image_extension)]
    exif = [Image(open(f, 'rb')) for f in source_files]
    
    for f, data in zip(source_files, exif):
        try:
            coord = get_decimal_coord_from_exif(data)
        except (AttributeError, KeyError):
            continue
        else:
            coord_dict[str(f)] = dict()
            coord_dict[str(f)]['img'] = f
            coord_dict[str(f)]['latitude'] = coord[0]
            coord_dict[str(f)]['longitude'] = coord[1]
            coord_dict[str(f)]['altitude'] = coord[2]
        # Also read date when photo was taken (if available)
        try:
            coord_dict[str(f)]['timestamp'] = data.datetime
        except (AttributeError, KeyError):
            print(f"Photo {f.name} does not contain datetime information.")
            coord_dict[str(f)]['timestamp'] = None
    
    return coord_dict

In [ ]:
image_path = Path("output")
resources = get_gps_data_from_images(image_path)

Photo MVIMG_20181126_132952-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181129_163040-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181126_101454-EFFECTS-edited.jpg does not contain datetime information.
Photo MVIMG_20181126_101454-EFFECTS.jpg does not contain datetime information.
Photo MVIMG_20181126_132952-EFFECTS-edited.jpg does not contain datetime information.
Photo MVIMG_20181129_163040-EFFECTS-edited.jpg does not contain datetime information.
Photo IMG_20181127_080204-EFFECTS.jpg does not contain datetime information.
Photo IMG_20181127_080204-EFFECTS-edited.jpg does not contain datetime information.


In [ ]:
resources

{'output/MVIMG_20181126_194541.jpg': {'img': Path('output/MVIMG_20181126_194541.jpg'),
  'latitude': 36.123004888888886,
  'longitude': -115.16538611111112,
  'altitude': 0.0,
  'timestamp': '2018:11:26 19:45:41'},
 'output/IMG_20181129_162952.jpg': {'img': Path('output/IMG_20181129_162952.jpg'),
  'latitude': 36.12946930555555,
  'longitude': -115.16462711111112,
  'altitude': 0.0,
  'timestamp': '2018:11:29 16:29:52'},
 'output/IMG_20181129_225115.jpg': {'img': Path('output/IMG_20181129_225115.jpg'),
  'latitude': 36.13149,
  'longitude': -115.16498319444445,
  'altitude': 0.0,
  'timestamp': '2018:11:29 22:51:15'},
 'output/IMG_20161125_215916.jpg': {'img': Path('output/IMG_20161125_215916.jpg'),
  'latitude': 45.569851194444446,
  'longitude': -73.52629461111111,
  'altitude': 0.0,
  'timestamp': '2016:11:25 22:59:17'},
 'output/IMG_20181129_130709.jpg': {'img': Path('output/IMG_20181129_130709.jpg'),
  'latitude': 36.129466388888886,
  'longitude': -115.164631,
  'altitude': 0.0,


In [ ]:
#| export

THUMB_PATH = "thumbnails"

def make_image_thumb(image: Image) -> str:
    """ makes thumbnails if they aren't created already or returns the path """
    thumbnail_path = Path(f"{THUMB_PATH}/{image.name}")
    if exists(thumbnail_path):
        return thumbnail_path  # don't bother reprocessing files
    MAX_SIZE = (300, 300)  # thumbnail size
    thumb = PIL.Image.open(image)
    thumb.thumbnail(MAX_SIZE)
    thumb.save(thumbnail_path)
    return thumbnail_path

def process_thumbnails(resources: dict)-> dict[str, dict]:
    """ Processes the thumbnails, and updates the resource dict with thumb location"""
    for k, v in resources.items():
        v['thumb'] = make_image_thumb(v['img'])
    return resources

In [ ]:
resources = process_thumbnails(resources)
resources

{'output/MVIMG_20181126_194541.jpg': {'img': Path('output/MVIMG_20181126_194541.jpg'),
  'latitude': 36.123004888888886,
  'longitude': -115.16538611111112,
  'altitude': 0.0,
  'timestamp': '2018:11:26 19:45:41',
  'thumb': Path('thumbnails/MVIMG_20181126_194541.jpg')},
 'output/IMG_20181129_162952.jpg': {'img': Path('output/IMG_20181129_162952.jpg'),
  'latitude': 36.12946930555555,
  'longitude': -115.16462711111112,
  'altitude': 0.0,
  'timestamp': '2018:11:29 16:29:52',
  'thumb': Path('thumbnails/IMG_20181129_162952.jpg')},
 'output/IMG_20181129_225115.jpg': {'img': Path('output/IMG_20181129_225115.jpg'),
  'latitude': 36.13149,
  'longitude': -115.16498319444445,
  'altitude': 0.0,
  'timestamp': '2018:11:29 22:51:15',
  'thumb': Path('thumbnails/IMG_20181129_225115.jpg')},
 'output/IMG_20161125_215916.jpg': {'img': Path('output/IMG_20161125_215916.jpg'),
  'latitude': 45.569851194444446,
  'longitude': -73.52629461111111,
  'altitude': 0.0,
  'timestamp': '2016:11:25 22:59:17'

In [ ]:
#| export
# create dataframe from extracted data

def make_dataframe(resources: dict) -> pd.DataFrame:
    df = pd.DataFrame(resources).T
    df['timestamp'] = pd.to_datetime(df.timestamp, format="%Y:%m:%d %H:%M:%S")
    df.sort_values('timestamp', inplace=True)

    df['color_mapping'] = np.linspace(0, 1, num=df.shape[0])
    return df

In [ ]:
df = make_dataframe(resources)
df

,img,latitude,longitude,altitude,timestamp,thumb,color_mapping
output/IMG_20161123_222428.jpg,output/IMG_20161123_222428.jpg,46.106831,-64.793534,0.0,2016-11-23 22:24:30,thumbnails/IMG_20161123_222428.jpg,0.00000
output/IMG_20161124_072147.jpg,output/IMG_20161124_072147.jpg,46.103096,-64.758859,0.0,2016-11-24 07:21:48,thumbnails/IMG_20161124_072147.jpg,0.00813
output/IMG_20161124_072509.jpg,output/IMG_20161124_072509.jpg,46.103096,-64.758859,0.0,2016-11-24 07:25:10,thumbnails/IMG_20161124_072509.jpg,0.01626
output/IMG_20161124_072520.jpg,output/IMG_20161124_072520.jpg,46.103102,-64.758921,0.0,2016-11-24 07:25:21,thumbnails/IMG_20161124_072520.jpg,0.02439
output/IMG_20161124_072526.jpg,output/IMG_20161124_072526.jpg,46.103102,-64.758921,0.0,2016-11-24 07:25:27,thumbnails/IMG_20161124_072526.jpg,0.03252
...,...,...,...,...,...,...,...
output/MVIMG_20181126_101454-EFFECTS.jpg,output/MVIMG_20181126_101454-EFFECTS.jpg,36.129464,-115.164684,0.0,NaT,thumbnails/MVIMG_20181126_101454-EFFECTS.jpg,0.96748
output/MVIMG_20181126_132952-EFFECTS-edited.jpg,output/MVIMG_20181126_132952-EFFECTS-edited.jpg,36.129461,-115.164656,0.0,NaT,thumbnails/MVIMG_20181126_132952-EFFECTS-edite...,0.97561
output/MVIMG_20181129_163040-EFFECTS-edited.jpg,output/MVIMG_20181129_163040-EFFECTS-edited.jpg,36.129469,-115.164627,0.0,NaT,thumbnails/MVIMG_20181129_163040-EFFECTS-edite...,0.98374
output/IMG_20181127_080204-EFFECTS.jpg,output/IMG_20181127_080204-EFFECTS.jpg,36.129465,-115.164636,0.0,NaT,thumbnails/IMG_20181127_080204-EFFECTS.jpg,0.99187


Colors need to be converted to hex to work with folium

In [ ]:
#| export
def rgba_to_hex(rgba: tuple[float, ...]):
    return ('#{:02X}{:02X}{:02X}').format(*rgba[:3])

maybe nab some text from exif to be able to populate some text in here

In [ ]:
#| export
def make_html(image: Path, thumbnail: Path) -> str:
    encoded_thumb = base64.b64encode(open(thumbnail, 'rb').read())
    html = f'<p> Here is some text with a link: <a href="http://localhost:8888/view/{image}" target="_blank">test link</a> <img src="data:image/jpeg;base64,{encoded_thumb.decode("UTF-8")}">'
    return html


def generate_map(df: pd.DataFrame) -> folium.Map:
    m = folium.Map()
    
    # get colour map
    cmap = matplotlib.cm.get_cmap('plasma')
    norm = matplotlib.colors.Normalize(vmin=0, vmax=1)

    # Add markers
    resolution, width, height = 75, 7, 3
    for img, thumb, lat, lon, col, date in zip(df.img,
                                               df.thumb,
                                               df.latitude.values, 
                                               df.longitude.values, 
                                               df.color_mapping.values, 
                                               df.timestamp.values):

        iframe = IFrame(make_html(img, thumb), width=(width*resolution)+20, height=(height*resolution)+20)
        popup = folium.Popup(iframe, max_width=2650)

        folium.CircleMarker(
            [lat, lon], 
            color=rgba_to_hex(cmap(col, bytes=True)), 
            fill_color=rgba_to_hex(cmap(col, bytes=True)),
            radius=6,
            popup = folium.Popup(iframe, max_width=2650),
            tooltip = np.datetime_as_string(date, unit='m')
            ).add_to(m)
    # create bounding box SW corner and NE corner, add 3 for padding
    sw = (df.latitude.max() + 3, df.longitude.min() - 3)
    ne = (df.latitude.min() - 3, df.longitude.max() + 3)
    m.fit_bounds([sw, ne])
    return m

In [ ]:
m = generate_map(df)
m

In [ ]:
from nbdev import nbdev_export
nbdev_export()